In [1]:
import igraph
import numpy as np
import pandas as pd
import pebble_game_algorithm.lib as lib
import logging

from functools import wraps
from time import time

In [2]:
FORMAT = '%(asctime)s %(levelname)s:%(message)s'
logging.basicConfig(level=logging.INFO, format=FORMAT, datefmt='%m/%d/%Y %I:%M:%S %p')

In [3]:
N = 100

In [4]:
def timing(func):
    """Decorator for timing a funciton."""
    @wraps(func)
    def wrap(*args, **kw):
        ts = time()
        result = func(*args, **kw)
        te = time()
        print('func:%r args:[%r, %r] took: %2.4f sec' % (func.__name__, args, kw, te-ts))
        return te-ts  # returns the total time spent.
    return wrap

In [5]:
# Open boundary
@timing
def obc(n=N):
    g = igraph.Graph.Lattice(dim=[n, n], circular=False)
    for idx, v in enumerate(g.vs):
        v["name"] = str(f"v_{idx}")
        v["free_pebble"] = 2

    edges = []
    for edge in g.es:
        source, target = edge.tuple
        edges.append((g.vs[source]["name"], g.vs[target]["name"]))
    g.delete_edges(edges)
    g.to_directed()

    for edge in edges:
        independent, stressed_cluster = lib.build_edge(g, edge)

    floppy_mode = len(g.vs) * 2 - len(g.es)
    print(f"Floppy mode number: {floppy_mode}")

In [6]:
profile = {}

In [7]:
for n in 2**np.array(range(8)):
    profile[n] = obc(n=n)

Floppy mode number: 2
func:'obc' args:[(), {'n': 1}] took: 0.0001 sec
Floppy mode number: 4
func:'obc' args:[(), {'n': 2}] took: 0.0001 sec
Floppy mode number: 8
func:'obc' args:[(), {'n': 4}] took: 0.0005 sec
Floppy mode number: 16
func:'obc' args:[(), {'n': 8}] took: 0.0041 sec
Floppy mode number: 32
func:'obc' args:[(), {'n': 16}] took: 0.0366 sec
Floppy mode number: 64
func:'obc' args:[(), {'n': 32}] took: 0.4683 sec
Floppy mode number: 128
func:'obc' args:[(), {'n': 64}] took: 7.0960 sec
Floppy mode number: 256
func:'obc' args:[(), {'n': 128}] took: 107.9588 sec


In [8]:
# Periodic boundary
@timing
def pbc():
    g = igraph.Graph.Lattice(dim=[N, N], circular=True)
    for idx, v in enumerate(g.vs):
        v["name"] = str(f"v_{idx}")
        v["free_pebble"] = 2

    edges = []
    for edge in g.es:
        source, target = edge.tuple
        edges.append((g.vs[source]["name"], g.vs[target]["name"]))
    g.delete_edges(edges)
    g.to_directed()

    for edge in edges:
        independent, stressed_cluster = lib.build_edge(g, edge)

    floppy_mode = len(g.vs) * 2 - len(g.es)
    print(f"Floppy mode number: {floppy_mode}")

In [9]:
pbc()

12/17/2021 12:21:24 PM INFO:Edge is redundant: ('v_98', 'v_9998')
12/17/2021 12:21:24 PM INFO:Edge is redundant: ('v_9900', 'v_9999')
12/17/2021 12:21:24 PM INFO:Edge is redundant: ('v_99', 'v_9999')


Floppy mode number: 3
func:'pbc' args:[(), {}] took: 286.9950 sec


286.9949700832367